In [12]:
import re

def process_file(input_file, output_file):
    with open(input_file, 'r') as file:
        lines = file.readlines()

    processed_lines = []
    Z = None  # Initialize Z value
    for line in lines:
        # Match lines containing Z value
        z_match = re.search(r'Z=(\d+\.\d+)', line)
        if z_match:
            Z = float(z_match.group(1))
            processed_lines.append(line)  # Keep the Z line unchanged
            continue

        # Add the new column name if the header line is detected
        if '&X_i,solar' in line:
            header_line = line.strip() + '  &X_i,0\n'
            processed_lines.append(header_line)
            continue

        # Process rows with data
        if Z is not None and re.search(r'&\w+', line):
            columns = line.split('&')
            if len(columns) > 8:  # Ensure it has the X_i,solar column
                try:
                    X_i_solar = float(columns[-1].strip())
                    X_i_0 = X_i_solar * Z / 0.0134
                    print(Z)
                    columns.append(f'{X_i_0:.6E}')
                    processed_lines.append('&'.join(columns) + '\n')
                except ValueError:
                    processed_lines.append(line)
            else:
                processed_lines.append(line)
        else:
            processed_lines.append(line)

    with open(output_file, 'w') as file:
        file.writelines(processed_lines)

# Input and output file paths
input_file = '/Users/adriana_work/Desktop/galIMF/yield_tables__2024/TABLE1-VW93ML_total_yields_scaled.txt'
output_file = '/Users/adriana_work/Desktop/galIMF/yield_tables__2024/TABLE1-VW93ML_total_yields_scaled_with_X_i_0_1.txt'

process_file(input_file, output_file)

0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02
0.02


In [11]:
import re

# File paths
input_file = "/Users/adriana_work/Desktop/galIMF/yield_tables__2024/TABLE1-VW93ML_total_yields_scaled_with_X_i_0.txt"
output_file = "/Users/adriana_work/Desktop/galIMF/yield_tables__2024/TABLE1-VW93ML_total_yields_scaled_with_X_i_0_total_yield.txt"

# Regular expression to match rows starting with '&'
row_pattern = re.compile(r"^&")

# Read the input file
with open(input_file, 'r') as file:
    lines = file.readlines()

# Process the file
updated_lines = []
for line in lines:
    if row_pattern.match(line):
        # Split the line into columns
        columns = line.split()

        # Extract relevant values
        try:
            p_i = float(columns[1])  # Net Yield
            M_ej = float(columns[2])  # MassExp
            X_i_0 = float(columns[-1])  # X_i,0 (last column)

            # Calculate Total yield
            total_yield = p_i + M_ej * X_i_0

            # Update the Total yield column (8th column, index 7)
            columns[7] = f"{total_yield:.6e}"
        except (ValueError, IndexError):
            # Skip rows with invalid data
            pass

        # Reconstruct the line
        line = " ".join(columns) + "\n"

    # Append the processed or unprocessed line
    updated_lines.append(line)

# Write the updated lines back to the file
with open(output_file, 'w') as file:
    file.writelines(updated_lines)

print("Total yield column updated successfully.")


Total yield column updated successfully.


In [ ]:
import pandas as pd

# Load AGB and CC SNe yield tables
agb_yields = pd.read_csv("yield_tables/agb_and_super_agb_and_massive_stars_K10_D13_LC18_R000 copy.txt")
cc_sne_yields = pd.read_csv("yield_tables/super_agb_doherty13.txt")

# Merge tables on common columns (e.g., element, metallicity)
combined_yields = pd.merge(agb_yields, cc_sne_yields, on=["Isotopes", "Metallicity"], how="outer", suffixes=("_AGB", "_CCSNe"))

# Sum yields for each element
combined_yields["Total_Yield"] = combined_yields["Yield_AGB"].fillna(0) + combined_yields["Yield_CCSNe"].fillna(0)

# Save the combined table
combined_yields.to_csv("combined_yields.csv", index=False)